# Covid19 Data Analysis

#### Overview
...

In [236]:
import datetime
import os
import io
import re
import json
import numpy as np
import pandas as pd
import scipy.stats
# import boto3
import pyarrow

In [195]:
def CONFIG():
    """"""
    _pid = 'analysis-covid19'
    _now = datetime.datetime.now()
    _cwd = os.getcwd()
    _data = os.path.join(_cwd,'data')
    
    _cfg = {
        'data': _data
        ,'eda': os.path.join(_cwd,'eda')
        ,'csse': os.path.join(_data,'csse-dly','{DT}.csv')
        ,'csse-us': os.path.join(_data,'csse-dly-us','{DT}.csv')
        ,'csse-ts': {
            'base': os.path.join(_data,'csse-ts','time_series_covid19_{STAT}_{RGN}.csv')
            ,'options': {
                'stat': ['confirmed','deaths','recovered']
                ,'rgn': ['global','US']
            }
            ,'errata': os.path.join(_data,'csse-ts','Errata.csv')
        }
        ,'who-ts': os.path.join(_data,'who-ts','who_covid_19_sit_rep_time_series.csv')
        ,'fips': os.path.join(_data,'csse-fips-lkup.csv')
    }
    
    return _cfg

CFG = CONFIG()

In [230]:
if __name__=='__main__':
    None